In [ ]:
# download
!git clone -b main --depth 1 https://github.com/khalil-research/PyEPO.git
# install
!pip install PyEPO/pkg/.

Cloning into 'PyEPO'...
remote: Enumerating objects: 119, done.
remote: Counting objects: 100% (119/119), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 119 (delta 18), reused 63 (delta 9), pack-reused 0
Receiving objects: 100% (119/119), 2.43 MiB | 17.93 MiB/s, done.
Resolving deltas: 100% (18/18), done.
Processing ./PyEPO/pkg
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 7.1 MB/s eta 0:00:00
  Created wheel for pyepo: filename=pyepo

## Build optModel

In [1]:
from coptpy import COPT
from pyepo.model.copt import optCoptModel

class myOptModel(optCoptModel):
    def _getModel(self):
        # ceate a model
        from coptpy import Envr
        m = Envr().createModel()
        # varibles
        x = m.addVars(5, nameprefix='x', vtype=COPT.BINARY)
        # sense
        m.setObjSense(COPT.MAXIMIZE)
        # constraints
        m.addConstr(3*x[0]+4*x[1]+3*x[2]+6*x[3]+4*x[4]<=12)
        m.addConstr(4*x[0]+5*x[1]+2*x[2]+3*x[3]+5*x[4]<=10)
        m.addConstr(5*x[0]+4*x[1]+6*x[2]+2*x[3]+3*x[4]<=15)
        return m, x

optmodel = myOptModel()

Auto-Sklearn cannot be imported.
Cardinal Optimizer v6.5.7. Build date Jul 28 2023
Copyright Cardinal Operations 2023. All Rights Reserved



## Problem Data

In [2]:
import torch
torch.manual_seed(42)

num_data = 1000 # number of data
num_feat = 5 # feature dimention
num_cost = 5 # cost dimention

# randomly generate data
x_true = torch.rand(num_data, num_feat) # feature
weight_true = torch.rand(num_feat, num_cost) # weight
bias_true = torch.randn(num_cost) # bias
noise = 0.5 * torch.randn(num_data, num_cost) # random noise
c_true = x_true @ weight_true + bias_true + noise # cost coef

In [ ]:
# split train test data
from sklearn.model_selection import train_test_split
x_train, x_test, c_train, c_test = train_test_split(x_true, c_true, test_size=200, random_state=42)

# build optDataset
from pyepo.data.dataset import optDataset
dataset_train = optDataset(optmodel, x_train, c_train)
dataset_test = optDataset(optmodel, x_test, c_test)

# build DataLoader
from torch.utils.data import DataLoader
batch_size = 32
loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

Optimizing for optDataset...


 69%|██████████████████████████████████████████████████████▉                         | 549/800 [00:10<00:04, 54.39it/s]

## Build Prediction Model

In [ ]:
import torch
from torch import nn

# build linear model
class LinearRegression(nn.Module):
    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(num_feat, num_cost)

    def forward(self, x):
        out = self.linear(x)
        return out

# init model
reg = LinearRegression()
# cuda
if torch.cuda.is_available():
    reg = reg.cuda()

## AutoGrad Module for Optimization

In [ ]:
import pyepo

# init SPO+ loss
spop = pyepo.func.SPOPlus(optmodel, processes=2)
# init PFY loss
pfy = pyepo.func.perturbedFenchelYoung(optmodel, n_samples=3, sigma=1.0, processes=2)
# init NCE loss
nce = pyepo.func.NCE(optmodel, processes=2, solve_ratio=0.05, dataset=dataset_train)

In [ ]:
  # set adam optimizer
  optimizer = torch.optim.Adam(reg.parameters(), lr=5e-3)

  # train mode
  reg.train()
  for epoch in range(5):
    # load data
    for i, data in enumerate(loader_train):
        x, c, w, z = data # feat, cost, sol, obj
        # cuda
        if torch.cuda.is_available():
            x, c, w, z = x.cuda(), c.cuda(), w.cuda(), z.cuda()
        # forward pass
        cp = reg(x)
        loss = spop(cp, c, w, z)
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # log
    regret = pyepo.metric.regret(reg, optmodel, loader_test)
    print("Loss: {:9.4f},  Regret: {:7.4f}%".format(loss.item(), regret*100))